# Quick Reference

**Prompt templates** are **predifined recipes for generating prompts for language models**.

A template may include:
- instrutions,
- few-shot examples,
- specific context and questions

appropriate for a given task.

LangChain provides tooling to create and work with prompt templates.

LangChain strives to create **model agnostic** templates to **make it easy to reuse existing templates across different language models**.

Typically, language models expect the prompt to be either:
- a string,
- a list of chat messages.

# `PromptTemplate`

Use `PromptTemplate` to create a template for a **string prompt**.

By default, `PromptTemplate` uses Python's `str.format` syntax for templating.

In [1]:
from langchain_core.prompts import PromptTemplate

In [2]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

In [3]:
prompt_template

PromptTemplate(input_variables=['adjective', 'content'], template='Tell me a {adjective} joke about {content}.')

In [4]:
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

The template **supports any number of variables**, including **no** variables.

In [5]:
prompt_template = PromptTemplate.from_template("Tell me a joke")

In [6]:
prompt_template

PromptTemplate(input_variables=[], template='Tell me a joke')

In [7]:
prompt_template.format()

'Tell me a joke'

You can create custom prompt templates that format the prompt in any way you want.

For more information, see [**Prompt Template Composition**](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/composition/) 👈

# `ChatPromptTeamplate`

The prompt to [**chat models**](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) is a list of [**chat messages**](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/)

each chat message is associated with:
- content,
- an additional parameter called `role`.

For example, in the OpenAI [**Chat Completions OpenAI**](https://platform.openai.com/docs/guides/text-generation/chat-completions-api), a chat message can be associated with an AI assistant, a human, or a system role.

Here's an example:

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [10]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are an helpful AI bot. Your name is {name}"),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

## 🔎 INTROSPECTION

In [11]:
# Inspecting chat_template's __repr__
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are an helpful AI bot. Your name is {name}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [12]:
chat_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are an helpful AI bot. Your name is {name}')),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))]

In [13]:
type(chat_template.messages)

list

In [17]:
type(chat_template[0])

langchain_core.prompts.chat.SystemMessagePromptTemplate

In [18]:
chat_template[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are an helpful AI bot. Your name is {name}'))

## BACK TO MAIN TOPIC

In [19]:
messages = chat_template.format_messages(name="Bob", user_input="What's your name?")

In [20]:
messages

[SystemMessage(content='You are an helpful AI bot. Your name is Bob'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content="What's your name?")]

Piping these formatted messages into LangChain's `ChatOpenAI` chat model class is roughly equivalent to the following, with usng the OpenAI client directly.

In [21]:
from openai import OpenAI

In [22]:
client = OpenAI()

In [23]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What's your name?"}
    ],
)

In [24]:
response

ChatCompletion(id='chatcmpl-9Ver1ais2FjdZ9KiUIdZrkWS7RJsB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='My name is Bob. How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1717332239, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=50, total_tokens=62))

The `ChatPromptTemplate.from_messages` static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.

For example, in addition to using the 2-tuple representation of (type, content) used above, you can pass in an instance of `MessagePromptTemplate` or `BaseMessage`.

In [25]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

In [26]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            "You are a helpful assistant that re-writes the user's text to "
            "sound more upbeat."
        )
    ),
    HumanMessagePromptTemplate.from_template("{text}"),
])

In [27]:
chat_template

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [30]:
messages = chat_template.format_messages(text="I don't like eating tasty things.")

In [31]:
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things.")]


# Message Prompts

LangChain provides different types of `MessagePromptTemplate`.

The most commonly used are:
- `AIMessagePromptTemplate`,
- `SystemMessagePromptTemplate`,
- `HumanMessagePromptTemplate`.

You can read more about [**the different types of messages**](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/)

In cases the chat model supports chat message with arbitrary role, you can use `ChatMessagePromptTemplate`, which allows to specify the `role` name.

In [32]:
from langchain_core.prompts import ChatMessagePromptTemplate

In [33]:
prompt = "May the {subject} be with you."

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role = "Jedi",
    template = prompt
)

In [34]:
type(chat_message_prompt)

langchain_core.prompts.chat.ChatMessagePromptTemplate

In [35]:
chat_message_prompt

ChatMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], template='May the {subject} be with you.'), role='Jedi')

In [36]:
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you.', role='Jedi')

# `MessagesPlaceholder`

LangChain also provides `MessagesPlaceholder`, which gives you full control of what messages to be rendered during formatting.

This can be useful:
- when you're uncertain of what role you should be using for your message prompt templates,
- when you wish to insert a list of messages during formatting.

In [37]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

In [38]:
human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)

In [39]:
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarize our conversation so far in {word_count} words.'))])

In [40]:
from langchain_core.messages import AIMessage, HumanMessage

In [41]:
human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
        """
)

chat_prompt.format_prompt(
    conversation = [human_message, ai_message], word_count="10"
).to_messages()

[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience        '),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

The full list of message prompt templates includes:
- `AIMessagePromptTemplate` for AI assistant messages,
- `SystemMessagePromptTemplate` for system messages,
- `HumanMMessagePromptTemplate` for user messages,
- `ChatMessagePromptTemplate` for messages with arbitrary role,
- `MessagePlaceholder` which accomodates a list of messages.

# LCEL

`PromptTemplate` and `ChatPromptTemplate` implement the [**Runnable Interface**](https://python.langchain.com/v0.1/docs/expression_language/interface/), the basic building bloc for the [**LangChain Expression Language (LCEL)**](https://python.langchain.com/v0.1/docs/expression_language/).

This means they support:
- `invoke`,
- `ainvoke`,
- `stream`,
- `astream`,
- `batch`,
- `abatch`,
- `astream_log`

calls.

- `PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`.
- `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [42]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})

In [43]:
type(prompt_val)

langchain_core.prompt_values.StringPromptValue

In [44]:
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

> There are convenient method to convert this object...

In [45]:
prompt_val.to_string()

'Tell me a funny joke about chickens.'

In [46]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funny joke about chickens.')]

In [47]:
type(prompt_val.to_messages())

list

In [48]:
type(prompt_val.to_messages()[0])

langchain_core.messages.human.HumanMessage

In [49]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            "You are a helpful assistant that re-writes the user's text to "
            "sound more upbeat."
        )
    ),
    HumanMessagePromptTemplate.from_template("{text}"),
])

chat_val = chat_template.invoke({"text":"I don't like eating tasty things."})

In [51]:
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things.")])

In [52]:
type(chat_val)

langchain_core.prompt_values.ChatPromptValue

In [53]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content="I don't like eating tasty things.")]

In [54]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: I don't like eating tasty things."